In [1]:
import numpy as np
import pandas as pd

In [2]:
import sqlite3

In [3]:
conn = sqlite3.connect('indicators_db.sqlite3')

In [4]:
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM sqlite_master', conn)

In [5]:
indicators

,type,name,tbl_name,rootpage,sql
0,table,org_indicators_2019,org_indicators_2019,2,CREATE TABLE org_indicators_2019 (\r\n\tID BIG...
1,table,org_indicators_2020_2022,org_indicators_2020_2022,490,CREATE TABLE org_indicators_2020_2022 (\r\n\tI...
2,view,v_org_indicators_all,v_org_indicators_all,0,CREATE VIEW v_org_indicators_all AS\r\nselect ...


In [6]:
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM org_indicators_2020_2022', conn)

In [7]:
indicators

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885
...,...,...,...,...
119995,339278,2022,Налоги,339380
119996,23511,2022,Число сотрудников,18
119997,23511,2022,Налоги,764187
119998,23511,2022,Выручка,22006160


## Задание 1
Для каждой организации из main.csv  
подтяните выручку и число сотрудников за 2020, 2021 и 2022.  
Если у организации нет данных по выручке или числу сотрудников за 2020 или 2022, то исключите их.  
Посчитайте дельту по выручке и сотрудникам между 2022 и 2020.  
Отсортируйте организации по убыванию выручки. Для организаций с одинаковой выручкой выполните сортировку по возрастанию ID.  
Названия колонок сделайте на русском языке.

In [3]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators = pd.read_sql('SELECT * FROM org_indicators_2020_2022', conn)
indicators

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885
...,...,...,...,...
119995,339278,2022,Налоги,339380
119996,23511,2022,Число сотрудников,18
119997,23511,2022,Налоги,764187
119998,23511,2022,Выручка,22006160


In [4]:
indicators = indicators[(indicators.INDICATOR_NAME == 'Выручка') | (indicators.INDICATOR_NAME == 'Число сотрудников')]

In [5]:
indicators = indicators.pivot_table('VALUE', ['ID'], ['INDICATOR_NAME', 'YEAR'])

In [6]:
indicators.columns = [col[0] + ' ' + str(col[1]) for col  in indicators.columns.values]

In [7]:
indicators.head()

,Выручка 2020,Выручка 2021,Выручка 2022,Число сотрудников 2020,Число сотрудников 2021,Число сотрудников 2022
ID,,,,,,
19630,31222228.0,21721921.0,NaN,NaN,15.0,9.0
19643,35670701.0,39125612.0,NaN,17.0,NaN,NaN
19647,34926614.0,23176004.0,27925131.0,16.0,15.0,NaN
19657,23410468.0,28065717.0,36074059.0,NaN,8.0,12.0
19677,NaN,30803884.0,25303592.0,17.0,9.0,NaN


In [8]:
indicators = indicators.dropna(subset=['Выручка 2020', 'Выручка 2022', 'Число сотрудников 2020', 'Число сотрудников 2022'])

In [9]:
indicators.loc[:, 'Дельта по выручке'] = indicators['Выручка 2022'] - indicators['Выручка 2020']
indicators.loc[:, 'Дельта по сотрудникам'] = indicators['Число сотрудников 2022'] - indicators['Число сотрудников 2020']

In [10]:
indicators = indicators.reset_index()

In [11]:
df_main = pd.read_csv('6_main.csv')
df_main

,ID,Name,Address
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15"
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ..."
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро..."
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро..."
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро..."
...,...,...,...
9995,192373,Рамен Клаб,"город Москва, Мясницкая улица, дом 16"
9996,204304,Папа Джонс,"город Москва, проспект Маршала Жукова, дом 59"
9997,301366,Up Sushi,"город Москва, город Московский, улица Москвити..."
9998,339278,Шаурма,"город Москва, улица Орджоникидзе, дом 12, стро..."


In [12]:
indicators = indicators.merge(df_main)

In [13]:
indicators = indicators.iloc[:, [0, -2, -1, 1, 2, 3, -4, 4, 5, 6, -3]]

In [14]:
indicators = indicators.rename(columns={'ID': 'Идентификатор', 'Name': 'Наименование', 'Address': 'Адрес'})

In [15]:
indicators = indicators.sort_values(['Дельта по выручке', 'Идентификатор'], ascending=[False, True])

In [16]:
indicators.to_csv('./6_5_1.csv', index=False)

## Задание 2
Коллеги уже пишут вам в чат и просят посчитать суммарную выручку по организациям за 2022 год: 357706, 198181, 27549.  
Через несколько минут приходит еще одно сообщение, просят добавить к ним еще четыре: 217895, 201139, 357706, 20105.  
Вы почти успели импортировать pandas и коллеги снова просят добавить к ним еще организаций из списка.  
Посчитайте, пожалуйста, суммарную выручку за 2022 год. Если у каких-то организаций нет выручки за 2022 год, то возьмите самую свежую доступную (за другой год). Ответ округлите до целого

In [105]:
df = pd.read_excel('6_Выручка_по_компаниям.xlsx', header=0, sheet_name='Лист1')

In [106]:
df = pd.concat([
    df, 
    pd.DataFrame([357706, 198181, 27549], columns=['Айдии']), 
    pd.DataFrame([217895, 201139, 357706, 20105], columns=['Айдии'])
])\
.drop_duplicates()\
.reset_index(drop=True)

In [107]:
df.columns = ['ID']

In [108]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators_20_22 = pd.read_sql('SELECT * FROM org_indicators_2020_2022', conn)
indicators_20_22.head()

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2020,Выручка,29337447
1,20988,2020,Оценка на Яндекс,2
2,20988,2020,Налоги,720554
3,20988,2020,Оценка на 2GIS,3
4,24828,2020,Выручка,36633885


In [109]:
indicators_20_22 = indicators_20_22[(indicators.INDICATOR_NAME == 'Выручка')]

In [110]:
indicators_20_22 = indicators_20_22.pivot_table('VALUE', ['ID'], ['INDICATOR_NAME', 'YEAR'])

In [111]:
indicators_20_22.columns = [col[0] + ' ' + str(col[1]) for col  in indicators_20_22.columns.values]

In [112]:
indicators_20_22.head()

,Выручка 2020,Выручка 2021,Выручка 2022
ID,,,
19630,31222228.0,21721921.0,NaN
19643,35670701.0,39125612.0,NaN
19647,34926614.0,23176004.0,27925131.0
19657,23410468.0,28065717.0,36074059.0
19677,NaN,30803884.0,25303592.0


In [113]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators_19 = pd.read_sql('SELECT * FROM org_indicators_2019', conn)
indicators_19.head()

,ID,YEAR,INDICATOR_NAME,VALUE
0,20988,2019,Число сотрудников,19
1,20988,2019,Оценка на Яндекс,2
2,20988,2019,Налоги,151143
3,20988,2019,Выручка,32914586
4,24828,2019,Оценка на 2GIS,1


In [114]:
indicators_19 = indicators_19[indicators_19.INDICATOR_NAME == 'Выручка']

In [115]:
indicators_19 = indicators_19.pivot_table('VALUE', ['ID'], ['INDICATOR_NAME', 'YEAR'])

In [116]:
indicators_19.columns = [col[0] + ' ' + str(col[1]) for col  in indicators_19.columns.values]

In [117]:
indicators_19.head()

,Выручка 2019
ID,
19630,20522186
19643,20577873
19677,34723333
19710,29753085
19721,32860968


In [119]:
indicators = indicators_19.join(indicators_20_22, how='outer')

In [120]:
indicators = indicators.fillna(axis=1, method='ffill')

In [124]:
df = df.merge(indicators, left_on='ID', right_index=True, how='left')

In [125]:
df['Выручка 2022'].sum()

462125109.0

## Задание 3
Если за 2020 год данных по сотрудникам нет, то возьмите за 2019 год.  
Выгрузите организации, где число сотрудников в 2020 году 15 и больше.  
Отсортируйте по возрастанию числа сотрудников. Если число сотрудников одинаковое, то организации должны идти по убыванию ID

In [17]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators_20_22 = pd.read_sql(
                                '''SELECT ID, VALUE 'Число сотрудников 2020'
                                FROM org_indicators_2020_2022
                                WHERE INDICATOR_NAME = 'Число сотрудников'
                                        AND YEAR = 2020'''
                        , conn)
indicators_20_22.head()

,ID,Число сотрудников 2020
0,358126,19
1,373052,16
2,303239,13
3,351220,6
4,211210,9


In [16]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators_19 = pd.read_sql(
                                '''SELECT ID, VALUE 'Число сотрудников 2019'
                                FROM org_indicators_2019 
                                WHERE INDICATOR_NAME = 'Число сотрудников'
                                        AND YEAR = 2019'''
                        , conn)
indicators_19.head()

,ID,Число сотрудников 2019
0,20988,19
1,24828,14
2,358126,12
3,303239,13
4,353273,15


In [21]:
indicators = indicators_20_22.merge(indicators_19, how='outer').fillna(axis=1, method='bfill')

In [12]:
df_main = pd.read_csv('6_main.csv')
df_main.head()

,ID,Name,Address
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15"
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ..."
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро..."
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро..."
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро..."


In [23]:
df = df_main.merge(indicators)

In [28]:
df = df[df['Число сотрудников 2020'] >= 15][['ID', 'Число сотрудников 2020']].sort_values(['Число сотрудников 2020', 'ID'], ascending=[True, False])

In [30]:
df.columns = ['Идентификатор', 'Сотрудники 2019_2020']

In [39]:
df['Сотрудники 2019_2020'] = df['Сотрудники 2019_2020'].astype('int')

In [42]:
df.to_csv('./6_5_3.csv', index=False)

## Задание 4
Найдите ТОП-10 организаций, которые за период 2019-2022 заплатили больше всего налогов и при этом платили налог все 4 года.  
В выгрузке должен быть идентификатор, наименование, адрес, налоги по годам и суммарный налог.  
Сортировка по убыванию уплаты налогов и по названию в алфавитном порядке (для равных по уплате налога).

In [25]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators_20_22 = pd.read_sql(
                                '''SELECT ID, YEAR, VALUE
                                FROM org_indicators_2020_2022
                                WHERE INDICATOR_NAME = 'Налоги'
                                '''
                        , conn)
indicators_20_22.head()

,ID,YEAR,VALUE
0,20988,2020,720554
1,24828,2020,729379
2,358126,2020,264739
3,303239,2020,102743
4,353273,2020,682232


In [26]:
indicators_20_22 = indicators_20_22.pivot_table('VALUE', 'ID', 'YEAR')
indicators_20_22.head()

YEAR,2020,2021,2022
ID,,,
19630,552636.0,741822.0,612311.0
19643,NaN,NaN,482734.0
19647,616861.0,770743.0,NaN
19657,NaN,NaN,649863.0
19677,500494.0,NaN,NaN


In [27]:
conn = sqlite3.connect('indicators_db.sqlite3')
with sqlite3.connect('indicators_db.sqlite3') as conn:
    indicators_19 = pd.read_sql(
                                '''SELECT ID, VALUE '2019'
                                FROM org_indicators_2019
                                WHERE INDICATOR_NAME = 'Налоги'
                                '''
                        , conn)
indicators_19.head()

,ID,2019
0,20988,151143
1,24828,225417
2,358126,738232
3,373052,799952
4,303239,766471


In [33]:
indicators_19 = indicators_19.set_index('ID')

In [36]:
indicators = indicators_19.join(indicators_20_22, how='outer')
indicators.head()

,2019,2020,2021,2022
ID,,,,
19630,NaN,552636.0,741822.0,612311.0
19643,NaN,NaN,NaN,482734.0
19647,NaN,616861.0,770743.0,NaN
19657,659466.0,NaN,NaN,649863.0
19677,359672.0,500494.0,NaN,NaN


In [56]:
indicators = indicators[indicators.notna().all(axis=1)]
indicators.head()

,2019,2020,2021,2022
ID,,,,
19688,332925.0,413374.0,794885.0,528160.0
19710,224787.0,532364.0,341558.0,500602.0
19721,726402.0,345211.0,325770.0,457974.0
19724,654272.0,645209.0,432603.0,633433.0
19728,715396.0,108681.0,327933.0,463617.0


In [57]:
indicators['Итого'] = indicators.sum(axis=1)

In [60]:
indicators = indicators.nlargest(10, 'Итого')

In [61]:
df_main = pd.read_csv('6_main.csv')
df_main.head()

,ID,Name,Address
0,20988,ГУЗ,"город Москва, улица Казакова, дом 15"
1,24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ..."
2,358126,Scarface,"Российская Федерация, город Москва, внутригоро..."
3,373052,Кулинариум,"Российская Федерация, город Москва, внутригоро..."
4,303239,One price coffee,"Российская Федерация, город Москва, внутригоро..."


In [63]:
df_main = df_main.set_index('ID')
df_main.head()

,Name,Address
ID,,
20988,ГУЗ,"город Москва, улица Казакова, дом 15"
24828,Кафе Халяль,"город Москва, 8-я улица Текстильщиков, дом 5, ..."
358126,Scarface,"Российская Федерация, город Москва, внутригоро..."
373052,Кулинариум,"Российская Федерация, город Москва, внутригоро..."
303239,One price coffee,"Российская Федерация, город Москва, внутригоро..."


In [65]:
df = df_main.join(indicators, how='right')

In [67]:
df = df.reset_index()

In [69]:
df.columns = [
        'Идентификатор', 
        'Наименование', 
        'Адрес', 
        'Налоги (2019 г.)',  
        'Налоги (2020 г.)',  
        'Налоги (2021 г.)',  
        'Налоги (2022 г.)',  
        'Налоги (Всего)'
]

In [72]:
df = df.sort_values(['Налоги (Всего)', 'Наименование'], ascending=[False, True])

In [74]:
df.to_csv('6_5_4.csv', index=False)

которые за период 2019-2022 заплатили больше всего налогов и при этом платили налог все 4 года.  
В выгрузке должен быть идентификатор, наименование, адрес, налоги по годам и суммарный налог.  
Сортировка по убыванию уплаты налогов и по названию в алфавитном порядке (для равных по уплате налога).  
Найдите ТОП-10 организаций, 